In [ ]:
# Setup to analyse an MDP Playground experiment
from mdp_playground.analysis import MDPP_Analysis
# Set dir_name to the location where the CSV files from running an experiment were saved
dir_name = '/home/rajanr/mdpp_14685068'
# Set exp_name to the name that was given to the experiment when running it
exp_name = 'ddpg_move_to_a_point_action_max_time_unit'
# Set the following to True to save PDFs of plots that you generate below
save_fig = True

In [ ]:
# Data loading
mdpp_analysis = MDPP_Analysis()
train_stats, eval_stats, train_curves, eval_curves, train_aucs, eval_aucs = mdpp_analysis.load_data(dir_name, exp_name, load_eval=False)

In [ ]:
# 1-D: Plots showing reward after 20k timesteps when varying a single meta-feature
# Plots across 10 runs: Training: with std dev across the runs
mdpp_analysis.plot_1d_dimensions(train_aucs, save_fig, bonferroni=False, err_bar='bootstrap')

In [ ]:
# Plots across 10 runs: Evaluation: with std dev across the runs
mdpp_analysis.plot_1d_dimensions(eval_stats, save_fig, train=False)

In [ ]:
# This and the next cell do the same as the previous 2 cells but plot episode mean lengths instead of episode reward
mdpp_analysis.plot_1d_dimensions(train_stats, save_fig, metric_num=-1)

In [ ]:
mdpp_analysis.plot_1d_dimensions(eval_stats, save_fig, train=False, metric_num=-1)

In [ ]:
# 2-D heatmap plots across 10 runs: Training runs: with std dev across the runs
# There seems to be a bug with matplotlib - x and y axes tick labels are not correctly set even though we pass them. Please feel free to look into the code and suggest a correction if you find it.
mdpp_analysis.plot_2d_heatmap(train_stats, save_fig)

In [ ]:
# 2-D heatmap plots across 10 runs: Evaluation runs: with std dev across the runs
mdpp_analysis.plot_2d_heatmap(eval_stats, save_fig, train=False)

In [ ]:
# Plot learning curves: Training: Each curve corresponds to a different seed for the agent
mdpp_analysis.plot_learning_curves(train_curves, save_fig)

In [ ]:
# Plot learning curves: Evaluation: Each curve corresponds to a different seed for the agent
mdpp_analysis.plot_learning_curves(eval_curves, save_fig, train=False)